# Evaluation

Get explanation accuracy for NetDissect and INVERT.

The MS COCO dataset requires the COCO API to be installed <https://github.com/cocodataset/cocoapi/tree/master>

In [ ]:
import os
import csv
import torch
import sympy
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Subset, DataLoader
from torchvision.models.segmentation import fcn_resnet50

from invert.evaluation import *
from invert.explainer import Invert
from invert.metrics import Metric

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
ROOT = "/root/path/"
BATCH_SIZE = 16
NUM_WORKERS = 2
QUANTILE = 0.005

In [ ]:
# Define Model
MODEL_NAME = "fcn_resnet50" # or "maskrcnn"

if MODEL_NAME == "maskrcnn":
    N_NEURONS = 91
    N_CLASSES = 91
    IMG_SIZE = 224
    SUBSET_SIZE = 25000
    DATA_NAME = "coco25k"
    IMAGES_PATH = f"{ROOT}COCO/images/train2017"
    ANN_PATH = f"{ROOT}COCO/annotations/instances_train2017.json"
    dataset_full = CocoDataset(root=IMAGES_PATH, annFile=ANN_PATH)
    dataset = Subset(dataset_full, range(SUBSET_SIZE))
    model = CustomMaskRCNN()
elif MODEL_NAME == "fcn_resnet50":
    N_NEURONS = 21
    N_CLASSES = 21
    IMG_SIZE = 520
    DATA_NAME = "coco-voc-val"
    IMAGES_PATH = f"{ROOT}COCO/images/val2017"
    ANN_PATH = f"{ROOT}COCO/annotations/instances_val2017.json"
    dataset = VocCocoDataset(root=IMAGES_PATH, annFile=ANN_PATH, transforms=SemanticSegmentation(resize_size=[IMG_SIZE,IMG_SIZE]))
    model = fcn_resnet50(pretrained=True, progress=False)

# Load dataset
dataloader = DataLoader(dataset,                        
                        batch_size=BATCH_SIZE,
                        collate_fn=collate_fn,
                        shuffle=False,
                        num_workers=NUM_WORKERS,
                        )
N_BATCHES = len(dataloader)
N_DATA = len(dataloader) * BATCH_SIZE

# Load model
model.to(device)
model.eval()

## 1. NetDissect

In [6]:
# Determine thresholds for neuron activations for each neuron.
quant = QuantileVector(depth=N_NEURONS, seed=1)
with torch.no_grad():
    for i, (image, mask, image_name, image_label) in tqdm(enumerate(dataloader)):
        image = image.to(device, dtype=torch.float32)
        if MODEL_NAME == "maskrcnn":
            output = model(image) # Bx91x224x224
        elif MODEL_NAME == "fcn_resnet50":
            output = model(image)["out"] # Bx21x520x520
            # normalize masks to probabilities
            output = torch.nn.functional.softmax(output, dim=1)
        batch = output.cpu().numpy()
        batch = np.transpose(batch, axes=(0, 2, 3, 1)).reshape( 
            -1, output.shape[1]
        )
        quant.add(batch)
ret = quant.readout(1000)[:, int(1000 * (1 - QUANTILE) - 1)]

# If all Quantiles set to 0.5
# ret = 0.5 * torch.ones(N_NEURONS)

0it [00:00, ?it/s]

313it [24:52,  4.77s/it]


In [10]:
# Get IoU scores
intersection_all = torch.zeros([N_BATCHES,N_NEURONS,N_CLASSES]).to(device)
union_all = torch.zeros([N_BATCHES,N_NEURONS,N_CLASSES]).to(device)
output_max_all = torch.zeros([N_DATA,N_CLASSES])
if not os.path.exists("assets/coco/"):
    os.makedirs("assets/coco/")

# Save image names with labels for INVERT
with open(f"assets/coco/{DATA_NAME}_labels.csv","w") as f1:
    writer=csv.writer(f1, delimiter=",",lineterminator="\n",)
    column_names = ["image_names"]
    column_names.extend(list(range(N_NEURONS)))
    writer.writerow(column_names)   

    counter = 0
    with torch.no_grad():
        for i, (image, mask, image_name, image_label) in tqdm(enumerate(dataloader)):
            for name, label in zip(image_name, image_label.to(device, dtype=torch.float32)):
                row = [name] + [str(int(i)) for i in label.tolist()]
                writer.writerow(row) 

            image = image.to(device, dtype=torch.float32)
            mask = mask.to(device, dtype=torch.float32)
            if MODEL_NAME == "maskrcnn":
                output = model(image) # Bx91x224x224
            elif MODEL_NAME == "fcn_resnet50":
                output = model(image)["out"] # Bx21x520x520
                # normalize masks to probabilities
                output = torch.nn.functional.softmax(output, dim=1)
            
            # Collect activations for INVERT
            output_max_all[counter:counter + image.shape[0],:] = torch.amax(output, dim=(2,3))

            binarized_masks = [(output[:, i, :, :] > ret[i]).float() for i in range(ret.shape[0])]
            binarized_masks = torch.stack(binarized_masks, dim=1)

            intersection = torch.mm(flatten_tens(binarized_masks), torch.transpose(flatten_tens(mask), 0, 1))
            intersection = intersection/(IMG_SIZE*IMG_SIZE)
            intersection_all[i, :, :] = intersection

            union = pairwise_sum(flatten_tens(binarized_masks), flatten_tens(mask))
            union = union/(IMG_SIZE*IMG_SIZE)
            union = union - intersection
            union_all[i, :, :] = union
            
            counter += image.shape[0]

# Save activations for INVERT
torch.save(output_max_all, f".invert/{MODEL_NAME}_output_max.pt")

intersection_all = torch.sum(intersection_all, dim=0) / N_BATCHES
union_all = torch.sum(union_all, dim=0) / N_BATCHES
iou = intersection_all / union_all

# Get accuracy
max_values, max_indices = torch.max(iou, dim=1)
matches = (max_indices == torch.arange(N_NEURONS).to(device))
if MODEL_NAME == "maskrcnn":
    bool_mask = torch.ones(N_NEURONS, dtype=torch.bool)
    bool_mask[empty_coco_classes] = 0
    matches = matches[bool_mask]
num_matches = matches.sum().item()
percentage = (num_matches/len(matches)) * 100

print(f"Number of matches: {int(num_matches)}/{len(matches)}")
print(f"Accuracy: {percentage:.2f}%")

0it [00:00, ?it/s]

313it [02:28,  2.10it/s]

Number of matches: 20/21
Accuracy: 95.24%


## 2.   INVERT

In [10]:
# Load data
df = pd.read_csv(f"assets/coco/{DATA_NAME}_labels.csv")

one_hot_labels = torch.tensor(df[df.columns[1:]].values)
if MODEL_NAME == "maskrcnn":
    data = coco_idx_class
elif MODEL_NAME == "fcn_resnet50":
    data = voc_idx_class

# Load activations
A = torch.load(f".invert/{MODEL_NAME}_output_max.pt") # CHANGE PATH
A = A[:one_hot_labels.size(0), :]

# Load INVERT
explainer = Invert(
        storage_dir=".invert/",
        device=device,
    )

explainer.load_activations(A=A,
                           Labels=one_hot_labels,
                           description=data,
                           dataset="coco"
                           )

In [ ]:
# Get INVERT explanations for all neurons
neuron_explain = {}
for i in tqdm(range(N_NEURONS)):
    explanation = explainer.explain_representation(i, 1, 1, Metric(), max_fraction=1)
    formula = repr(explanation[0]["formula"])
    if "~" not in formula:
        neuron_explain[i] = int(formula)
    else:
        neuron_explain[i] = str(formula)

In [12]:
# Get accuracy
count = 0
for pred, target in neuron_explain.items():
    if MODEL_NAME == "maskrcnn":
        if target in empty_coco_classes:
            continue    
    if pred == target:
        count +=1
if MODEL_NAME == "maskrcnn":
    total_class = len(neuron_explain) - len(empty_coco_classes)
elif MODEL_NAME == "fcn_resnet50":
    total_class = len(neuron_explain)
percentage = (count/total_class) * 100

print(f"Number of matches: {int(count)}/{total_class}")
print(f"Accuracy: {percentage:.2f}%")

Number of matches: 20/21
Accuracy: 95.24%
